In [1]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from copy import *

In [38]:
np.random.seed(10)

data = {
    "Ücret (TL)": np.random.randint(700000, 2000000, size=100),
    "Oda Sayısı": np.random.randint(1, 6, size=100),
    "Semtin Demografik Yapısı": np.random.randint(1, 5, size=100),
    "Suç Oranı": np.random.uniform(0, 1, size=100),
    "Bina Dayanıklılık Puanı": np.random.randint(1, 101, size=100),
    "Kat Sayısı": np.random.randint(1, 11, size=100),
    "Manzara": np.random.randint(1, 4, size=100),
    "Okul Kalitesi": np.random.randint(1, 6, size=100),
    "Alışveriş Merkezine Uzaklık (km)": np.random.uniform(0, 10, size=100),
    "Metroya Uzaklık (km)": np.random.uniform(0, 10, size=100),
}

data = pd.DataFrame(data)
data


,Ücret (TL),Oda Sayısı,Semtin Demografik Yapısı,Suç Oranı,Bina Dayanıklılık Puanı,Kat Sayısı,Manzara,Okul Kalitesi,Alışveriş Merkezine Uzaklık (km),Metroya Uzaklık (km)
0,1748548,5,2,0.726099,89,8,1,5,2.228636,6.499367
1,1317841,1,1,0.268116,9,9,1,5,2.127267,7.554071
2,1854171,3,4,0.345196,87,3,1,2,6.279880,8.853143
3,1504336,4,4,0.353722,76,1,3,1,6.332761,2.530527
4,1857021,4,1,0.263114,73,4,3,5,3.032790,2.279938
...,...,...,...,...,...,...,...,...,...,...
95,1926880,4,3,0.367716,20,9,2,5,4.369589,9.662300
96,1474815,3,3,0.669686,79,8,1,4,2.702327,9.426875
97,1704901,1,1,0.406751,100,3,3,2,9.689634,5.892653
98,737065,5,3,0.356950,33,8,3,2,5.898090,2.295584


In [39]:
#data = pd.read_csv("ev_liste.csv" , sep=";")
data.insert(0, "Ev ID", range(1, len(data)+1))
data.to_csv("evler.csv", index=False)
data = pd.read_csv("evler.csv" , sep=",")
df_normalized = data.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))
populasyon = len(data)



In [40]:

def generate_attr (size):
    return list(np.random.uniform(low=0.0, high=1.0, size=size))
def generate_population (size, attr_size):
    return [generate_attr(attr_size) for i in range (size)]


In [41]:
#Hedef fonksiyonu tanımlanması
s_min = np.min(df_normalized, axis=0)
s_max = np.max(df_normalized, axis=0)
importance_weight = [1,4,9,3,2,8,5,6,10,7]
weights = [-0.3, 0.15, 0.03, -0.3, 0.23, 0.05, - 0.13, 0.12, -0.02,  -0.08]
def fitness(attr):
    fitness = 0
    for j, col in enumerate(df_normalized.values.tolist()[0][1:]):
        fitness += weights[j] * ((col-s_min[j]) / (s_max[j] - s_min[j]) )
    return fitness

In [42]:
def select_parent(population, scores):
    return population[np.random.choice(range(len(population)), p=scores/np.sum(scores))]

In [43]:
def crossover(parent1, parent2):
    crossover_point = np.random.randint(1, len(parent1))
    child = parent1[:crossover_point] + parent2[crossover_point:]
    return child


In [44]:
def mutation(attr, probability=0.5):
    for i in range(len(attr)):
        if np.random.uniform() < probability:
            attr[i] = np.random.uniform()
    return attr

In [45]:
def genetic_algorithm(population_size, generations):
    population = generate_population(population_size, 10)
    for i in range(generations):
        scores = [fitness(attr) for attr in population]
        best_attr = population[scores.index(max(scores))]
        parent1 = select_parent(population, scores)
        parent2 = select_parent(population, scores)
        child = crossover(parent1, parent2)
        child = mutation(child)
        new_population = []
        for i in range(population_size):
            new_population.append(select_parent(population, scores))
        population = [child] + new_population[1:]
    return best_attr

In [46]:
# En iyi özellik vektörüne sahip olan evin indeksini bulma
best_attr = genetic_algorithm(50, 100)
print("Best attribute:", best_attr)

Best attribute: [0.3059537787037877, 0.3886563451073852, 0.2866595368721877, 0.9826545769798094, 0.23744431941102184, 0.8080341766151046, 0.8001813897634292, 0.3393778185254388, 0.9247766311923584, 0.18529634035453513]


In [47]:
df_normalized = np.array(df_normalized)

best_distance = float('inf')
best_match_index = -1
for i, row in enumerate(df_normalized):
    distance = np.linalg.norm(row[1:] - np.array(best_attr))
    if distance < best_distance:
        best_distance = distance
        best_match_index = i

print("Best match index:", best_match_index)

Best match index: 56


In [48]:
for index, row in data.iterrows():
    if (index == best_match_index-1):
        formatted_row = [f'{value:.2f}' for value in row]
        print(f"Row {index}: {formatted_row}")
        

Row 55: ['56.00', '1175496.00', '1.00', '1.00', '0.79', '87.00', '10.00', '2.00', '4.00', '9.67', '4.82']


HILL-CLIMB

Hill-Climbing algoritması şu adımlarla çalışır:

* Başlangıç noktasını belirle: Rastgele bir ev seçilir.
* Komşu çözümleri hesapla: Mevcut evin özelliklerini biraz değiştirerek yeni bir ev oluştur.
* Yeni evin kalitesini hesapla: Yeni evin kalitesi (hedef fonksiyonu) ölçülür.
* Mevcut evle yeni evi karşılaştır: Yeni evin kalitesi mevcut evden daha yüksekse, yeni ev mevcut çözüm olarak seçilir ve adımlar tekrarlanır. Aksi takdirde, mevcut ev çözüm olarak kalır.
* Belli bir sayıda adım veya belirli bir zaman içinde en iyi sonucu veren çözüm bulunana kadar 2-4. adımlar tekrarlanır.

In [49]:
#Hedef fonksiyonu tanımlanması
s_min = np.min(df_normalized, axis=0)
s_max = np.max(df_normalized, axis=0)
def fitness_func(attr):
    importance_weight = [1,4,9,3,2,8,5,6,10,7]
    weights = [-0.3, 0.23, 0.19, 0.15, 0.13, 0.12, -0.08, 0.05, 0.03, 0.02]
    fitness = 0
    for j, col in enumerate(attr):
        fitness += weights[j] * ((col-s_min[j]) / (s_max[j] - s_min[j]) )
    return fitness

In [50]:

current_solution = df_normalized[random.randint(0, len(df_normalized) - 1)][1:]
best_score = fitness_func(current_solution)

In [51]:
max_steps = 100
step = 0
# Hill-Climbing algoritması
while step < max_steps:
    new_solution = copy(current_solution)
    # Bir rastgele özelliği değiştir
    new_solution = mutation(new_solution)
    # Yeni çözümü değerlendir
    new_score = fitness_func(new_solution)
    # Karşılaştır ve en iyi çözümü güncelle
    if new_score > best_score:
        best_solution = new_solution
        best_score = new_score
    if new_score > fitness_func(current_solution):
        current_solution = new_solution
    step += 1


In [52]:
df_normalized = np.array(df_normalized)

best_distance = float('inf')
best_match_index1 = -1
for i, row in enumerate(df_normalized):
    distance = np.linalg.norm(row[1:] - np.array(best_solution))
    if distance < best_distance:
        best_distance = distance
        best_match_index1 = i

print("Best match index:", best_match_index1)

Best match index: 86


In [53]:
for index, row in data.iterrows():
    if (index == best_match_index1-1):
        formatted_row = [f'{value:.2f}' for value in row]
        print(f"Row {index}: {formatted_row}")

Row 85: ['86.00', '739697.00', '3.00', '1.00', '0.96', '66.00', '9.00', '1.00', '2.00', '0.16', '9.20']


In [54]:
print(fitness_func(df_normalized[best_match_index-1][1:]))

0.28360632623720294


In [55]:
print(fitness_func(df_normalized[best_match_index1-1][1:]))

0.4816426920842535
